In [ ]:
#In prep

In [ ]:
x = grouped.drop(['DateTime','TowerID','TagID','POINT_X','POINT_Y'],axis= 1)
y = grouped['POINT_Y']

In [ ]:
seed = 38

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=seed)

In [ ]:
model = LinearRegression()
model.fit(X_train,y_train)

In [ ]:
predictions = model.predict(X_test)
print('mean_squared_error : ', mean_squared_error(y_test, predictions))
print('mean_absolute_error : ', mean_absolute_error(y_test, predictions))

In [ ]:
predictions

In [ ]:
plt.scatter(y_test, predictions)